# Refinamiento de un modelo (T5) usando librerias HuggingFace



#### Seleccione un modelo preentrenado para refinarlo

Elija el modelo T5 para texto en Ingles.

In [ ]:
model_checkpoint = 

#### Cargar la base de datos

Para acortar el tiempo de ejecución de este ejemplo, carguemos solo las primeras 5000 instancias de la base de datos seleccionada

In [ ]:
from datasets import load_dataset

dataset = load_dataset("", split="train[:5000]")

#### Preparación del conjunto de datos

Para preparar la base de datos para el entrenamiento y evaluación, cree los diccionarios necesarios. Estos serán útiles al realizar inferencias y para obtener información de metadatos:

In [ ]:
labels = dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

A continuacion, cargue el procesador de texto usado en el preentrenamiendo del modelo

In [ ]:
def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch[""] = [train_transforms() for  in example_batch[""]]
    return example_batch


def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch[""] = [val_transforms() for  in example_batch[""]]
    return example_batch

Divide la base de datos para entrenamiento y validacion:

In [ ]:
splits = dataset.train_test_split(test_size=0.1)
train_ds = splits["train"]
val_ds = splits["test"]

Finalmente, configure las funciones de transformación para la base de datos:

In [ ]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

#### Preparacion del modelo

Antes de cargar el modelo, definamos una función auxiliar para verificar la cantidad total de parámetros que tiene un modelo, así como cuántos de ellos se pueden entrenar.

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Params entrenables: {trainable_params} || Totalidad: {all_param} || entrenables%: {100 * trainable_params / all_param:.2f}"
    )

Es importante inicializar el modelo original correctamente, ya que se utilizará como base para crear el PeftModel que realmente ajustará.

In [ ]:
from transformers import , TrainingArguments, Trainer

model = .from_pretrained(
    model_checkpoint,

    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

Antes de crear un PeftModel, puede verificar la cantidad de parámetros entrenables en el modelo original:

In [ ]:
print_trainable_parameters(model)

A continuación, utilice `get_peft_model` para ajustar el modelo base de modo que se agreguen matrices de "actualización" en los lugares respectivos.

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="none",
    modules_to_save=["classifier"],
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

Analicemos lo que está pasando aquí. Para usar LoRA, debe especificar los módulos de destino en `LoraConfig` para que `get_peft_model()` sepa qué módulos dentro de nuestro modelo deben modificarse con matrices LoRA. En este ejemplo, solo nos interesa apuntar a las matrices de consulta y valor de los bloques de atención del modelo base. Dado que los parámetros correspondientes a estas matrices se "llaman" "consulta" y "valor" respectivamente, los especificamos en consecuencia en el argumento target_modules de LoraConfig.

También especificamos `module_to_save`. Después de envolver el modelo base con `get_peft_model()` junto con la configuración, obtenemos un nuevo modelo en el que solo se pueden entrenar los parámetros LoRA (las llamadas "matrices de actualización") mientras que los parámetros previamente entrenados se mantienen congelados. Sin embargo, queremos que los parámetros del clasificador también se entrenen al ajustar el modelo base en nuestro conjunto de datos personalizado. Para garantizar que los parámetros del clasificador también estén entrenados, especificamos `module_to_save`. Esto también garantiza que estos módulos se serialicen junto con los parámetros entrenables de LoRA cuando se utilizan utilidades como `save_pretrained()` y` push_to_hub()`.

Esto es lo que significan los otros parámetros:

`r`: La dimensión utilizada por las matrices de actualización de LoRA.
`alfa`: factor de escala.
`bias`: especifica si los parámetros bias (tendencia) deben entrenarse. Ninguno indica que no se entrenará ninguno de los parámetros bias.
`r` y `alpha` juntos controlan la cantidad total de parámetros finales entrenables cuando se usa LoRA, lo que le brinda la flexibilidad de equilibrar el equilibrio entre el rendimiento final y la eficiencia informática.

Al observar la cantidad de parámetros entrenables, puede ver cuántos parámetros estamos entrenando realmente. Dado que el objetivo es lograr un ajuste fino eficiente en los parámetros, debería esperar ver menos parámetros entrenables en lora_model en comparación con el modelo original, que de hecho es el caso aquí.

#### Definiendo parametros de entrenamiento

Para ajustar el modelo, utilice `Trainer`. Acepta varios argumentos que puedes ajustar usando `TrainingArguments`.

In [ ]:
from transformers import TrainingArguments, Trainer


model_name = model_checkpoint.split("/")[-1]
batch_size = 128

args = TrainingArguments(
    f"{model_name}-finetuned-lora-food101",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-3,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    fp16=True,
    num_train_epochs=5,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    label_names=["labels"],
)

En comparación con los métodos que no son PEFT, puede utilizar un tamaño de lote mayor ya que hay menos parámetros para entrenar. También puedes establecer una tasa de aprendizaje mayor que la normal (`1e-5` por ejemplo).

Potencialmente, esto también puede reducir la necesidad de realizar costosos experimentos de ajuste de hiperparámetros.

#### Preparar métrica de evaluación

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Calcula la precision en el subconjunto de predicciones."""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

La función Compute_metrics toma una tupla con nombre como entrada: `predictions`, que son los logits del modelo como matrices Numpy, y `label_ids`, que son las etiquetas de verdad fundamental como matrices Numpy.

#### Definiendo la función de intercalación

`Trainer` utiliza una función de intercalación para recopilar un lote de ejemplos de capacitación y evaluación y prepararlos en un formato que sea aceptable para el modelo subyacente.

In [ ]:
import torch


def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

#### Entrenamiento y evaluacion

Reúna todo: modelo, argumentos de entrenamiento, datos, función de intercalación, etc. Luego, ¡comience el entrenamiento!

In [ ]:
trainer = Trainer(
    lora_model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)
train_results = trainer.train()

trainer.evaluate(val_ds)

#### Ejecuta una inferencia

Veamos cómo cargar los parámetros actualizados de LoRA junto con nuestro modelo base para inferencia. Cuando envuelve un modelo base con PeftModel, las modificaciones se realizan in situ. Para mitigar cualquier inquietud que pueda surgir de las modificaciones locales, inicialice el modelo base tal como lo hizo antes y construya el modelo de inferencia.

In [ ]:
from peft import PeftConfig, PeftModel


config = PeftConfig.from_pretrained(repo_name)
model = AutoModelForImageClassification.from_pretrained(
    config.base_model_name_or_path,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
# Load the LoRA model
inference_model = PeftModel.from_pretrained(model, repo_name)



¡Finalmente, ejecute la inferencia!

In [ ]:
with torch.no_grad():
    outputs = inference_model(**encoding)
    logits = outputs.logits